In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from sentencebert_missingpositionprediction import *

I0529 08:26:52.554205 140176275728192 file_utils.py:41] PyTorch version 1.4.0 available.


In [3]:
import numpy as np
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [4]:
device = torch.device('cuda')

In [5]:
# train
train_dataset = ROCStoriesDataset_random_missing(data_path = "../data/rocstories_completion_train.csv")

# dev
val_dataset = ROCStoriesDataset_with_missing(data_path = "../data/rocstories_completion_dev.csv")

# test
test_dataset = ROCStoriesDataset_with_missing(data_path = "../data/rocstories_completion_test.csv")

In [ ]:
%%capture

batch_size = 32
block_size = 32

sentbertmodel = SentenceTransformer('bert-base-nli-mean-tokens')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

# --- model --- 
model = MissingPisitionPredictionModel(sentbertmodel, device, ContextEncoder="PoolContextEncoder").to(device)

# --- DataLoader ---|
collate_fn = lambda data: collate(data, tokenizer, block_size=block_size, device=device)
train_iterator = DataLoader(
    train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size, collate_fn=collate_fn,
)    
valid_iterator = DataLoader(
    val_dataset, sampler=SequentialSampler(val_dataset), batch_size=batch_size, collate_fn=collate_fn,
)    
test_iterator = DataLoader(
    test_dataset, sampler=SequentialSampler(test_dataset), batch_size=batch_size, collate_fn=collate_fn,
)    

TRG_PAD_IDX = tokenizer.pad_token_id
START_ID = tokenizer.cls_token_id
mpe_criterion = nn.CrossEntropyLoss()

## load model and get accuracy

In [7]:
accuracy_list = {}

In [8]:
model_dirs = ["<PATH to trained model directories>"]
model_name = "sentbert-positionestimation_model.pt"

In [9]:
for model_dir in model_dirs:
    print(os.path.basename(os.path.dirname(model_dir)))

    model_path = os.path.join(model_dir, model_name)
    model = MissingPisitionPredictionModel(sentbertmodel, device, ContextEncoder="PoolContextEncoder").to(device)
    model.load_state_dict(torch.load(model_path))
    test_loss, test_acc = evaluate(model, test_iterator, mpe_criterion)
    
    accuracy_list[os.path.basename(os.path.dirname(model_dir))] = test_acc

20191230_trial15_30epochs_pool_seed42


Iteration: 100%|██████████| 307/307 [02:32<00:00,  2.01it/s]


20191230_trial16_30epochs_pool_seed1234


Iteration: 100%|██████████| 307/307 [02:32<00:00,  2.01it/s]


20191230_trial17_30epochs_pool_seed99


Iteration: 100%|██████████| 307/307 [02:33<00:00,  2.01it/s]


20191230_trial18_30epochs_pool_seed613


Iteration: 100%|██████████| 307/307 [02:32<00:00,  2.01it/s]


20191230_trial19_30epochs_pool_seed1000


Iteration: 100%|██████████| 307/307 [02:32<00:00,  2.01it/s]


In [10]:
accuracy_list

{'20191230_trial15_30epochs_pool_seed42': 0.34647801302931597,
 '20191230_trial16_30epochs_pool_seed1234': 0.3540105863192183,
 '20191230_trial17_30epochs_pool_seed99': 0.35052117263843646,
 '20191230_trial18_30epochs_pool_seed613': 0.34594055374592836,
 '20191230_trial19_30epochs_pool_seed1000': 0.3506514657980456}

In [11]:
import json    

In [12]:
with open("./result/mpp_baseline_test_accuracy.json", "w") as f:
    json.dump(accuracy_list, f, indent=4)

# mean, std

In [13]:
import statistics as st

In [14]:
values = np.array(list(accuracy_list.values()))

In [15]:
values = values * 100

In [16]:
values

array([34.6478013 , 35.40105863, 35.05211726, 34.59405537, 35.06514658])

In [17]:
print('statistics')

print("mean")
print(st.mean(values))

print("std")
print(st.stdev(values))

statistics
mean
34.9520358306189
std
0.33359423918936915
